In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium
from classes.models import LinearClassifier
from datasets import load_datasets, load_mnist, load_cifar10, load_cifar100
from sklearn.svm import LinearSVC
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load datasets

In [2]:
data, samplers, pair_samplers, triplet_samplers = load_mnist(augment=True)

MNIST loaded in 39 seconds
Fashion MNIST loaded in 39 seconds


### Testing function

In [12]:
def test_model(model, settings, data, feature_samplers, classification_samplers, num_features, num_steps, lr, keep_prob, reg):
    tf.reset_default_graph()
    f_train, f_test = feature_samplers
    c_train, c_test = classification_samplers
    (x_train, y_train), (x_test, y_test) = data
    _, H, W, C = x_train.shape
    
    x = tf.placeholder(tf.float32, shape=[None, H, W, C])
    y = tf.placeholder(tf.int64, shape=[None])
    dropout = tf.placeholder(tf.float32)
    training = tf.placeholder(tf.bool)
    m = model(x=x, y=y, settings=settings, num_chan=C, num_features=num_features, 
              lr=lr, reg=reg, dropout=dropout, training=training)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training features // num_features: %g, lr: %g, reg: %g"%(num_features, lr, reg))
        train_time = time.time()
        for step in range(num_steps):        
            x_, y_ = f_train.sample(200)
            sess.run(m.optimize, feed_dict={x:x_, y:y_, dropout:keep_prob, training:True})     
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([m.loss, m.acc], feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training features // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = f_test.sample(5000)
        test_error = 1 - sess.run(m.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        svc = LinearSVC(random_state=0)
        x_, y_ = c_train.sample(20000)
        features_train = sess.run(m.features, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        svc.fit(features_train, y_)
        features_test = sess.run(m.features, feed_dict={x:x_test, y:y_test, dropout:1.0, training:False})
        print("classification accuracy: {:.4f}\n".format(svc.score(features_test, y_test)))

# MNIST Triplet Tests

In [5]:
for lr in np.random.uniform(0.03, 0.1, 4):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['mnist'], samplers=triplet_samplers['mnist'],
               num_features=10, num_steps=15000, lr=lr, keep_prob=0.8, reg=0.001)

begin training features // num_features: 10, lr: 0.0747453, reg: 0.001
	step 0: train loss 1.22649, train error 0.46
	step 5000: train loss 0.0441183, train error 0
	step 10000: train loss 0.0209183, train error 0
end training features // time elapsed: 116.8834 s
test set error: 0.0038 // time elapsed: 0.1766 s
classification accuracy: 0.9842

begin training features // num_features: 10, lr: 0.0986515, reg: 0.001
	step 0: train loss 1.23039, train error 0.5
	step 5000: train loss 0.0324496, train error 0
	step 10000: train loss 0.0169188, train error 0
end training features // time elapsed: 116.6316 s
test set error: 0.0030 // time elapsed: 0.1702 s
classification accuracy: 0.9860

begin training features // num_features: 10, lr: 0.063485, reg: 0.001
	step 0: train loss 1.23051, train error 0.485
	step 5000: train loss 0.0491716, train error 0
	step 10000: train loss 0.0258126, train error 0
end training features // time elapsed: 117.5176 s
test set error: 0.0034 // time elapsed: 0.183

In [11]:
for lr in np.random.uniform(0.01, 0.1, 5):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['mnist'], samplers=triplet_samplers['mnist'],
               num_features=20, num_steps=15000, lr=lr, keep_prob=0.8, reg=0.001)

begin training features // num_features: 20, lr: 0.0373034, reg: 0.001
	step 0: train loss 1.23043, train error 0.495
	step 5000: train loss 0.108515, train error 0.005
	step 10000: train loss 0.040082, train error 0
end training features // time elapsed: 116.0268 s
test set error: 0.0028 // time elapsed: 0.2633 s
classification accuracy: 0.9843

begin training features // num_features: 20, lr: 0.0692201, reg: 0.001
	step 0: train loss 1.22494, train error 0.47
	step 5000: train loss 0.0463578, train error 0
	step 10000: train loss 0.0199733, train error 0
end training features // time elapsed: 116.1093 s
test set error: 0.0042 // time elapsed: 0.2693 s
classification accuracy: 0.9840

begin training features // num_features: 20, lr: 0.0759431, reg: 0.001
	step 0: train loss 1.24072, train error 0.545
	step 5000: train loss 0.0406525, train error 0
	step 10000: train loss 0.0190406, train error 0
end training features // time elapsed: 116.8040 s
test set error: 0.0048 // time elapsed: 

In [6]:
for lr in np.random.uniform(0.02, 0.1, 5):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['mnist'], samplers=triplet_samplers['mnist'],
               num_features=40, num_steps=15000, lr=lr, keep_prob=0.8, reg=0.001)

begin training features // num_features: 40, lr: 0.0363715, reg: 0.001
	step 0: train loss 1.24914, train error 0.5
	step 5000: train loss 0.113362, train error 0.00999999
	step 10000: train loss 0.043342, train error 0
end training features // time elapsed: 116.5782 s
test set error: 0.0020 // time elapsed: 0.1912 s
classification accuracy: 0.9841

begin training features // num_features: 40, lr: 0.0826486, reg: 0.001
	step 0: train loss 1.24419, train error 0.455
	step 5000: train loss 0.0374918, train error 0
	step 10000: train loss 0.0201124, train error 0
end training features // time elapsed: 116.9268 s
test set error: 0.0036 // time elapsed: 0.1858 s
classification accuracy: 0.9839

begin training features // num_features: 40, lr: 0.023884, reg: 0.001
	step 0: train loss 1.24599, train error 0.455
	step 5000: train loss 0.13431, train error 0
	step 10000: train loss 0.0996143, train error 0.005
end training features // time elapsed: 117.4132 s
test set error: 0.0036 // time elap

# Augmented Data

In [8]:
for lr in np.random.uniform(0.03, 0.15, 5):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['mnist'], samplers=triplet_samplers['mnist'],
               num_features=10, num_steps=15000, lr=lr, keep_prob=0.8, reg=0.001)

begin training features // num_features: 10, lr: 0.059262, reg: 0.001
	step 0: train loss 1.22897, train error 0.49
	step 4000: train loss 0.0858287, train error 0
	step 8000: train loss 0.0605886, train error 0.005
	step 12000: train loss 0.0523309, train error 0.00999999
end training features // time elapsed: 118.3798 s
test set error: 0.0036 // time elapsed: 0.1799 s
classification accuracy: 0.9811

begin training features // num_features: 10, lr: 0.0951218, reg: 0.001
	step 0: train loss 1.22956, train error 0.47
	step 4000: train loss 0.065812, train error 0.005
	step 8000: train loss 0.0501976, train error 0
	step 12000: train loss 0.0351119, train error 0
end training features // time elapsed: 118.7211 s
test set error: 0.0032 // time elapsed: 0.1908 s
classification accuracy: 0.9823

begin training features // num_features: 10, lr: 0.100889, reg: 0.001
	step 0: train loss 1.23, train error 0.495
	step 4000: train loss 0.0859322, train error 0.015
	step 8000: train loss 0.056828

In [ ]:
for lr in np.random.uniform(0.03, 0.15, 5):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['mnist'], samplers=triplet_samplers['mnist'],
               num_features=20, num_steps=15000, lr=lr, keep_prob=0.8, reg=0.001)

begin training features // num_features: 20, lr: 0.112168, reg: 0.001
	step 0: train loss 1.23382, train error 0.5
	step 4000: train loss 0.0593057, train error 0.005
	step 8000: train loss 0.0369204, train error 0
	step 12000: train loss 0.0377043, train error 0.005
end training features // time elapsed: 118.2453 s
test set error: 0.0016 // time elapsed: 0.1988 s
classification accuracy: 0.9871

begin training features // num_features: 20, lr: 0.141733, reg: 0.001
	step 0: train loss 1.23479, train error 0.415
	step 4000: train loss 0.0511741, train error 0
	step 8000: train loss 0.0419123, train error 0
	step 12000: train loss 0.0410957, train error 0
end training features // time elapsed: 117.9538 s
test set error: 0.0026 // time elapsed: 0.2057 s
classification accuracy: 0.9861

begin training features // num_features: 20, lr: 0.145708, reg: 0.001
	step 0: train loss 1.23608, train error 0.495
	step 4000: train loss 0.0679675, train error 0.005
	step 8000: train loss 0.0527129, tra

In [13]:
for lr in np.random.uniform(0.03, 0.15, 3):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['mnist'], samplers=triplet_samplers['mnist'],
               num_features=40, num_steps=15000, lr=lr, keep_prob=0.8, reg=0.001)

begin training features // num_features: 40, lr: 0.0732618, reg: 0.001
	step 0: train loss 1.24144, train error 0.495
	step 4000: train loss 0.0929606, train error 0.00999999
	step 8000: train loss 0.0561131, train error 0.005
	step 12000: train loss 0.0330865, train error 0
end training features // time elapsed: 117.4174 s
test set error: 0.0036 // time elapsed: 0.2119 s
classification accuracy: 0.9825

begin training features // num_features: 40, lr: 0.142971, reg: 0.001
	step 0: train loss 1.22403, train error 0.415
	step 4000: train loss 0.0654998, train error 0.005
	step 8000: train loss 0.0436373, train error 0
	step 12000: train loss 0.0384467, train error 0
end training features // time elapsed: 118.8584 s
test set error: 0.0022 // time elapsed: 0.2229 s
classification accuracy: 0.9866

begin training features // num_features: 40, lr: 0.111092, reg: 0.001
	step 0: train loss 1.24466, train error 0.46
	step 4000: train loss 0.0806207, train error 0.005
	step 8000: train loss 0.0

# MNIST Pair Tests (Adagrad optimizer)

In [11]:
for lr in np.random.uniform(0.007, 0.012, 3):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['mnist'], samplers=pair_samplers['mnist'],
               num_features=10, num_steps=15000, lr=lr, keep_prob=0.8, reg=0)

begin training features // num_features: 10, lr: 0.00847161, reg: 0
	step 0: train loss 1.62533, train error 0.89
	step 5000: train loss 0.952284, train error 0.47
	step 10000: train loss 0.462537, train error 0.17
end training features // time elapsed: 87.0650 s
test set error: 0.1072 // time elapsed: 0.2684 s
classification accuracy: 0.8635

begin training features // num_features: 10, lr: 0.00901295, reg: 0
	step 0: train loss 1.77281, train error 0.915
	step 5000: train loss 0.999498, train error 0.495
	step 10000: train loss 0.568107, train error 0.225
end training features // time elapsed: 86.9414 s
test set error: 0.1082 // time elapsed: 0.2252 s
classification accuracy: 0.8864

begin training features // num_features: 10, lr: 0.00651539, reg: 0
	step 0: train loss 1.86173, train error 0.94
	step 5000: train loss 0.780267, train error 0.33
	step 10000: train loss 0.490319, train error 0.175
end training features // time elapsed: 86.8976 s
test set error: 0.1128 // time elapsed: 

In [ ]:
for lr in np.random.uniform(0.01, 0.03, 3):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['mnist'], samplers=pair_samplers['mnist'],
               num_features=20, num_steps=15000, lr=lr, keep_prob=0.8, reg=0)

begin training features // num_features: 20, lr: 0.0132497, reg: 0
	step 0: train loss 1.61059, train error 0.89
	step 5000: train loss 0.632083, train error 0.24
	step 10000: train loss 0.351145, train error 0.105
end training features // time elapsed: 86.0507 s
test set error: 0.0278 // time elapsed: 0.2836 s
classification accuracy: 0.9573

begin training features // num_features: 20, lr: 0.0126081, reg: 0
	step 0: train loss 1.62727, train error 0.89
	step 5000: train loss 0.519724, train error 0.165
	step 10000: train loss 0.190258, train error 0.065
end training features // time elapsed: 86.7726 s
test set error: 0.0410 // time elapsed: 0.2450 s
classification accuracy: 0.9526

begin training features // num_features: 20, lr: 0.00610331, reg: 0
	step 0: train loss 1.42406, train error 0.86
	step 5000: train loss 0.850974, train error 0.41
	step 10000: train loss 0.470849, train error 0.18
end training features // time elapsed: 86.5777 s
test set error: 0.1062 // time elapsed: 0.2

In [5]:
for lr in np.random.uniform(0.01, 0.035, 4):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['mnist'], samplers=pair_samplers['mnist'],
               num_features=40, num_steps=15000, lr=lr, keep_prob=0.8, reg=0)

begin training features // num_features: 40, lr: 0.018455, reg: 0
	step 0: train loss 1.80684, train error 0.89
	step 5000: train loss 0.608959, train error 0.245
	step 10000: train loss 0.177772, train error 0.06
end training features // time elapsed: 387.6101 s
test set error: 0.0248 // time elapsed: 0.1941 s
classification accuracy: 0.9606

begin training features // num_features: 40, lr: 0.0305185, reg: 0
	step 0: train loss 2.31007, train error 0.9
	step 5000: train loss 0.666152, train error 0.295
	step 10000: train loss 0.12812, train error 0.05
end training features // time elapsed: 384.0874 s
test set error: 0.0314 // time elapsed: 0.1761 s
classification accuracy: 0.8875

begin training features // num_features: 40, lr: 0.0221891, reg: 0
	step 0: train loss 1.81611, train error 0.9
	step 5000: train loss 0.0708081, train error 0.02
	step 10000: train loss 0.0252054, train error 0.00999999
end training features // time elapsed: 386.5518 s
test set error: 0.0124 // time elapsed

# Augmented Data

In [31]:
for lr in np.random.uniform(0.01, 0.03, 3):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['mnist'], samplers=pair_samplers['mnist'],
               num_features=10, num_steps=15000, lr=lr, keep_prob=0.8, reg=0)

begin training features // num_features: 10, lr: 0.0108642, reg: 0
	step 0: train loss 1.29718, train error 0.885
	step 4000: train loss 1.00266, train error 0.875
	step 8000: train loss 0.979311, train error 0.48
	step 12000: train loss 0.764198, train error 0.31
end training features // time elapsed: 87.5860 s
test set error: 0.1960 // time elapsed: 0.1538 s
classification accuracy: 0.8151

begin training features // num_features: 10, lr: 0.0100808, reg: 0
	step 0: train loss 1.26411, train error 0.925
	step 4000: train loss 1.0026, train error 0.87
	step 8000: train loss 1.00269, train error 0.84
	step 12000: train loss 0.857726, train error 0.38
end training features // time elapsed: 86.7527 s
test set error: 0.2712 // time elapsed: 0.1470 s
classification accuracy: 0.7620

begin training features // num_features: 10, lr: 0.0150876, reg: 0
	step 0: train loss 1.39692, train error 0.925
	step 4000: train loss 1.00072, train error 0.57
	step 8000: train loss 0.697347, train error 0.2

In [28]:
for lr in np.random.uniform(0.015, 0.03, 3):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['mnist'], samplers=pair_samplers['mnist'],
               num_features=20, num_steps=15000, lr=lr, keep_prob=0.8, reg=0)

begin training features // num_features: 20, lr: 0.0214675, reg: 0
	step 0: train loss 1.4666, train error 0.885
	step 4000: train loss 1.0032, train error 0.79
	step 8000: train loss 0.514572, train error 0.205
	step 12000: train loss 0.405674, train error 0.13
end training features // time elapsed: 86.0124 s
test set error: 0.0764 // time elapsed: 0.0944 s
classification accuracy: 0.8781

begin training features // num_features: 20, lr: 0.0292262, reg: 0
	step 0: train loss 1.60217, train error 0.9
	step 4000: train loss 0.764863, train error 0.285
	step 8000: train loss 0.316047, train error 0.12
	step 12000: train loss 0.141271, train error 0.055
end training features // time elapsed: 85.6583 s
test set error: 0.0388 // time elapsed: 0.1003 s
classification accuracy: 0.9009

begin training features // num_features: 20, lr: 0.0237202, reg: 0
	step 0: train loss 1.88621, train error 0.92
	step 4000: train loss 0.885426, train error 0.4
	step 8000: train loss 0.383373, train error 0.1

In [30]:
for lr in np.random.uniform(0.015, 0.04, 3):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['mnist'], samplers=pair_samplers['mnist'],
               num_features=40, num_steps=15000, lr=lr, keep_prob=0.8, reg=0)

begin training features // num_features: 40, lr: 0.0165854, reg: 0
	step 0: train loss 1.82771, train error 0.89
	step 4000: train loss 1.00289, train error 0.95
	step 8000: train loss 1.00333, train error 0.73
	step 12000: train loss 0.640527, train error 0.255
end training features // time elapsed: 86.7912 s
test set error: 0.0906 // time elapsed: 0.1186 s
classification accuracy: 0.8403

begin training features // num_features: 40, lr: 0.0220736, reg: 0
	step 0: train loss 1.63185, train error 0.895
	step 4000: train loss 1.01056, train error 0.495
	step 8000: train loss 0.435688, train error 0.155
	step 12000: train loss 0.25372, train error 0.065
end training features // time elapsed: 86.5853 s
test set error: 0.0478 // time elapsed: 0.1417 s
classification accuracy: 0.8853

begin training features // num_features: 40, lr: 0.0263975, reg: 0
	step 0: train loss 1.97354, train error 0.9
	step 4000: train loss 0.735703, train error 0.3
	step 8000: train loss 0.325682, train error 0.1

# Fashion MNIST Tests

In [12]:
for lr in np.random.uniform(0.05, 0.15, 3):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=triplet_samplers['fashion_mnist'],
               num_features=10, num_steps=20000, lr=lr, keep_prob=0.8, reg=0.002)

begin training features // num_features: 10, lr: 0.119433, reg: 0.002
	step 0: train loss 1.45781, train error 0.45
	step 4000: train loss 0.0760131, train error 0.00999999
	step 8000: train loss 0.0914871, train error 0.025
	step 12000: train loss 0.0776445, train error 0.03
	step 16000: train loss 0.0666515, train error 0.02
end training features // time elapsed: 157.1605 s
test set error: 0.0242 // time elapsed: 0.2092 s
classification accuracy: 0.8881

begin training features // num_features: 10, lr: 0.117851, reg: 0.002
	step 0: train loss 1.46418, train error 0.51
	step 4000: train loss 0.114691, train error 0.04
	step 8000: train loss 0.100385, train error 0.025
	step 12000: train loss 0.0666172, train error 0.02
	step 16000: train loss 0.0521798, train error 0.015
end training features // time elapsed: 157.4858 s
test set error: 0.0294 // time elapsed: 0.1716 s
classification accuracy: 0.8885

begin training features // num_features: 10, lr: 0.105822, reg: 0.002
	step 0: train 

In [25]:
for lr in np.random.uniform(0.05, 0.15, 3):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=triplet_samplers['fashion_mnist'],
               num_features=20, num_steps=20000, lr=lr, keep_prob=0.8, reg=0.002)

begin training features // num_features: 20, lr: 0.119088, reg: 0.002
	step 0: train loss 1.47016, train error 0.375
	step 4000: train loss 0.120975, train error 0.03
	step 8000: train loss 0.0895732, train error 0.015
	step 12000: train loss 0.0833746, train error 0.025
	step 16000: train loss 0.0366519, train error 0.005
end training features // time elapsed: 157.2065 s
test set error: 0.0318 // time elapsed: 0.3166 s
classification accuracy: 0.8867

begin training features // num_features: 20, lr: 0.0526343, reg: 0.002
	step 0: train loss 1.47449, train error 0.49
	step 4000: train loss 0.148847, train error 0.035
	step 8000: train loss 0.112621, train error 0.04
	step 12000: train loss 0.114331, train error 0.03
	step 16000: train loss 0.0320912, train error 0.005
end training features // time elapsed: 157.4039 s
test set error: 0.0358 // time elapsed: 0.3205 s
classification accuracy: 0.8744

begin training features // num_features: 20, lr: 0.0691768, reg: 0.002
	step 0: train los

In [5]:
for lr in np.random.uniform(0.005, 0.02, 6):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=pair_samplers['fashion_mnist'],
               num_features=10, num_steps=20000, lr=lr, keep_prob=0.8, reg=0.002)

begin training features // num_features: 10, lr: 0.00865784, reg: 0.002
	step 0: train loss 2.11515, train error 0.92
	step 4000: train loss 1.05817, train error 0.305
	step 8000: train loss 0.862917, train error 0.195
	step 12000: train loss 0.643841, train error 0.135
	step 16000: train loss 0.532855, train error 0.115
end training features // time elapsed: 114.8823 s
test set error: 0.1532 // time elapsed: 0.0822 s
classification accuracy: 0.7387

begin training features // num_features: 10, lr: 0.0136432, reg: 0.002
	step 0: train loss 1.63396, train error 0.925
	step 4000: train loss 0.753372, train error 0.155
	step 8000: train loss 0.559513, train error 0.145
	step 12000: train loss 0.450207, train error 0.095
	step 16000: train loss 0.616267, train error 0.125
end training features // time elapsed: 114.8808 s
test set error: 0.1176 // time elapsed: 0.0814 s
classification accuracy: 0.7666

begin training features // num_features: 10, lr: 0.0153829, reg: 0.002
	step 0: train los

In [6]:
for lr in np.random.uniform(0.005, 0.03, 6):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=pair_samplers['fashion_mnist'],
               num_features=20, num_steps=20000, lr=lr, keep_prob=0.8, reg=0)

begin training features // num_features: 20, lr: 0.0217065, reg: 0
	step 0: train loss 1.44929, train error 0.87
	step 4000: train loss 0.531581, train error 0.12
	step 8000: train loss 0.411968, train error 0.135
	step 12000: train loss 0.388416, train error 0.13
	step 16000: train loss 0.379726, train error 0.135
end training features // time elapsed: 114.2696 s
test set error: 0.1080 // time elapsed: 0.1059 s
classification accuracy: 0.7981

begin training features // num_features: 20, lr: 0.0218056, reg: 0
	step 0: train loss 1.45932, train error 0.865
	step 4000: train loss 0.521368, train error 0.175
	step 8000: train loss 0.368553, train error 0.135
	step 12000: train loss 0.311548, train error 0.085
	step 16000: train loss 0.230428, train error 0.105
end training features // time elapsed: 114.8639 s
test set error: 0.0986 // time elapsed: 0.1260 s
classification accuracy: 0.7963

begin training features // num_features: 20, lr: 0.00960272, reg: 0
	step 0: train loss 1.5977, tra

# Augmentation

In [33]:
for lr in np.random.uniform(0.05, 0.15, 3):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=triplet_samplers['fashion_mnist'],
               num_features=10, num_steps=20000, lr=lr, keep_prob=0.8, reg=0.002)

begin training features // num_features: 10, lr: 0.0775914, reg: 0.002
	step 0: train loss 1.4636, train error 0.465
	step 4000: train loss 0.0818809, train error 0.005
	step 8000: train loss 0.062792, train error 0.005
	step 12000: train loss 0.0509089, train error 0
	step 16000: train loss 0.0497464, train error 0.00999999
end training features // time elapsed: 158.0652 s
test set error: 0.0038 // time elapsed: 0.2867 s
classification accuracy: 0.9822

begin training features // num_features: 10, lr: 0.0960076, reg: 0.002
	step 0: train loss 1.46496, train error 0.455
	step 4000: train loss 0.0709629, train error 0.005
	step 8000: train loss 0.0626883, train error 0.005
	step 12000: train loss 0.0399753, train error 0
	step 16000: train loss 0.0537192, train error 0.00999999
end training features // time elapsed: 157.5807 s
test set error: 0.0030 // time elapsed: 0.2301 s
classification accuracy: 0.9853

begin training features // num_features: 10, lr: 0.101034, reg: 0.002
	step 0: t

In [34]:
for lr in np.random.uniform(0.05, 0.15, 3):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=triplet_samplers['fashion_mnist'],
               num_features=20, num_steps=20000, lr=lr, keep_prob=0.8, reg=0.002)

begin training features // num_features: 20, lr: 0.120133, reg: 0.002
	step 0: train loss 1.47892, train error 0.55
	step 4000: train loss 1.00111, train error 0.455
	step 8000: train loss 1, train error 0.545
	step 12000: train loss 1, train error 1
	step 16000: train loss 1, train error 1
end training features // time elapsed: 158.5085 s
test set error: 1.0000 // time elapsed: 0.2476 s
classification accuracy: 0.1135

begin training features // num_features: 20, lr: 0.0675542, reg: 0.002
	step 0: train loss 1.47008, train error 0.41
	step 4000: train loss 0.0699491, train error 0
	step 8000: train loss 0.0626706, train error 0.005
	step 12000: train loss 0.0531413, train error 0
	step 16000: train loss 0.0446168, train error 0.005
end training features // time elapsed: 156.6717 s
test set error: 0.0044 // time elapsed: 0.2838 s
classification accuracy: 0.9835

begin training features // num_features: 20, lr: 0.0835517, reg: 0.002
	step 0: train loss 1.47784, train error 0.495
	step 4

In [31]:
for lr in np.random.uniform(0.01, 0.15, 4):
    test_model(model=ConvModelSmall, settings=Triplet_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=triplet_samplers['fashion_mnist'],
               num_features=40, num_steps=20000, lr=lr, keep_prob=0.8, reg=0.001)

begin training features // num_features: 40, lr: 0.13256, reg: 0.001
	step 0: train loss 1.24831, train error 0.525
	step 1000: train loss 0.299273, train error 0.05
	step 2000: train loss 0.214023, train error 0.055
	step 3000: train loss 0.18164, train error 0.04
	step 4000: train loss 0.217303, train error 0.07
	step 5000: train loss 0.159904, train error 0.045
	step 6000: train loss 0.114296, train error 0.02
	step 7000: train loss 0.13144, train error 0.045
	step 8000: train loss 0.144272, train error 0.045
	step 9000: train loss 0.130618, train error 0.05
	step 10000: train loss 0.147199, train error 0.025
	step 11000: train loss 0.134921, train error 0.03
	step 12000: train loss 0.111659, train error 0.03
	step 13000: train loss 0.105656, train error 0.025
	step 14000: train loss 0.103824, train error 0.03
	step 15000: train loss 0.115541, train error 0.04
	step 16000: train loss 0.119241, train error 0.025
	step 17000: train loss 0.178198, train error 0.065
	step 18000: train l

In [50]:
for lr in np.random.uniform(0.001, 0.01, 2):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=pair_samplers['fashion_mnist'],
               num_features=10, num_steps=10000, lr=lr, keep_prob=0.8, reg=0.002)

begin training features // num_features: 10, lr: 0.00292767, reg: 0.002
	step 0: train loss 2.48459, train error 0.91
	step 4000: train loss 1.45283, train error 0.935
	step 8000: train loss 1.43023, train error 0.88
end training features // time elapsed: 261.6228 s
test set error: 0.8892 // time elapsed: 0.5404 s
classification accuracy: 0.3858

begin training features // num_features: 10, lr: 0.00690873, reg: 0.002
	step 0: train loss 1.63593, train error 0.92
	step 4000: train loss 1.42279, train error 0.96
	step 8000: train loss 1.37834, train error 0.94
end training features // time elapsed: 260.0945 s
test set error: 0.9040 // time elapsed: 0.5677 s
classification accuracy: 0.4068



In [51]:
for lr in np.random.uniform(0.001, 0.01, 2):
    test_model(model=ConvModelSmall, settings=Pair_Hinge_Settings, 
               data=data['fashion_mnist'], samplers=pair_samplers['fashion_mnist'],
               num_features=20, num_steps=10000, lr=0.01, keep_prob=0.8, reg=0.002)

begin training features // num_features: 20, lr: 0.01, reg: 0.002
	step 0: train loss 1.72006, train error 0.88
	step 4000: train loss 1.40857, train error 0.935
	step 8000: train loss 1.3476, train error 0.915
end training features // time elapsed: 262.1665 s
test set error: 0.8974 // time elapsed: 0.5301 s
classification accuracy: 0.4749

begin training features // num_features: 20, lr: 0.01, reg: 0.002
	step 0: train loss 2.26366, train error 0.89
	step 4000: train loss 1.40845, train error 0.9
	step 8000: train loss 1.34737, train error 0.885
end training features // time elapsed: 261.5377 s
test set error: 0.8514 // time elapsed: 0.5608 s
classification accuracy: 0.5800

